In [5]:
import jdc
import math

import cv2
import os

import numpy as np
from numpy import linalg as LA

import torch
from torch import linalg as LAT

import scipy
from scipy import linalg as LAS
from scipy.stats import unitary_group
from scipy.stats import ortho_group

from tensorly import unfold
import imutils

In [4]:
class KarcherLSR:
    def __init__(self):
        self.name = "Least Squares Regression"
    
    def __unfold(self,X, mode):
        z, x, y = X.shape

        if mode == 0:
            G = np.zeros((x,0), dtype=float)
            for i in range(z):
                G = np.concatenate((G, X[i,:,:]), axis=1)

        if mode == 1:
            G = np.zeros((y,0), dtype=float)
            for i in range(z):
                G = np.concatenate((G, X[i,:,:].T), axis=1)

        if mode == 2:
            G = np.zeros((z,0), dtype=float)
            for i in range(y):
                G = np.concatenate((G, X[:,:,i]), axis=1)
        return  G

    def __hosvd(self,A):
        X1 = __unfold(A, mode=0)
        X2 = __unfold(A, mode=1)
        X3 = __unfold(A, mode=2)


        u, s, vh1 = np.linalg.svd(X1,full_matrices=False)
        u, s, vh2 = np.linalg.svd(X2,full_matrices=False)
        u, s, vh3 = np.linalg.svd(X3,full_matrices=False)


        return [vh1.transpose(),vh2.transpose(),vh3.transpose()]

    def __my_hosvd(self,A):
        dim = unfold(A,0)
        dim2 = unfold(A,1)
        dim3 = unfold(A,2)

        _,_,vh1 = LAS.svd(dim,full_matrices=False)
        _,_,vh2 = LAS.svd(dim2,full_matrices=False)
        _,_,vh3 = LAS.svd(dim3,full_matrices=False)

        return [np.matrix.getH(vh1),np.matrix.getH(vh2),np.matrix.getH(vh3)]

    def __stiefel_to_grassmann(self,stief):
        grass = []
        for i in range(len(stief)):
            orth = ortho_group.rvs(stief[i].shape[1])
            grass.append(np.dot(stief[i],orth))
        return grass

    def __get_geodisic_distance(self,theta):
        return LA.norm(np.sin(theta))

    def __get_canonical_angles(self,fm1,fm2):
        #return scipy.linalg.subspace_angles(vlist1, vlist2)
        assert(fm1.shape[1] == fm2.shape[1])
        S = LAS.svdvals(np.dot(fm.T,fm2))
        T= np.zeros(fm1.shape[1])
        for k in range(fm1.shape[1]):
            if S[k] < 10 ** -8:
                T[k] = np.sqrt(2*(1-S[k]))
            else:
                T[k] = np.real(np.arccos(S[k]))
        return np.flip(T)
        

    def __get_prod_geod(self,gras_a,gras_b):

        assert(len(gras_a) == len(gras_b))
        canon = []
        for i in range(len(gras_a)):
            canon.append(self.__get_canonical_angles(gras_a[i],gras_b[i]))
        cart_prod = [(a,b,c) for a in canon[0] for b in canon[1] for c in canon[2]]
        sin = np.sin(np.array(cart_prod))
        return LA.norm(sin,ord='fro')

    def __log_map(self,x,y):
        
        #temp = (eye(m)-X*X')*Y*inv(X'*Y);
        r = x.shape[0]
        temp = (np.eye(r) - x@x.T)@y@LA.inv(x.T@y)
        [U,S,V] = LA.svd(temp,full_matrices = False)
        theta = np.arctan(S)
        return U@np.diag(theta)@V.T
    
    def __exp_map(self,point,weights):
        [U,S,V] = LA.svd(weights,full_matrices = False);
        y = point@V@np.diag(np.cos(S)) + U@np.diag(np.sin((S)))
        return y
    
    def __on_man(self,pt1):
        r,c = pt1.shape
        M = pt1.T@pt1
        I = np.eye(c)

        if(LA.norm(M-I,ord ='fro') > 10**-10):
            print("Not in Man")
        else:
            print("In Man")
            
    def __karcher_mean(self,base_point,class_data):
        
        #self.__on_man(base_point)
    
        r,c = base_point.shape
        err = 99999999
        num_iters = 0
        tolerance = 0.1
        
        training_weight = np.zeros((r,c))
        
        while(err > tolerance and num_iters < 50):
            
            for i in range(len(class_data)):
                training_weight = training_weight + self.__log_map(base_point,class_data[i])

            training_weight = np.divide(training_weight,len(class_data))
#             print(LA.norm(training_weight))

            updated_base = self.__exp_map(base_point,training_weight)
            #print(updated_base.shape,base_point.shape)
            err = LA.norm(self.__get_canonical_angles(updated_base,base_point))
            
            base_point = updated_base
            num_iters+=1
            
        print("CONVERGED after ",num_iters," iterations")    
            
        return base_point
  

    def fit(self,X_train,y_train):
            
        data = []
        for class_num in range(len(X_train)):
            xy,yt,xt = [],[],[]
            for data2 in range(len(X_train[class_num])):
                video = X_train[class_num][data2]
                fac_man = self.__stiefel_to_grassmann(self.__my_hosvd(video))
                xy.append(fac_man[0])
                yt.append(fac_man[1])
                xt.append(fac_man[2])
            xy_prime = self.__karcher_mean(xy[0],np.array(xy))
            yt_prime = self.__karcher_mean(yt[0],np.array(yt))
            xt_prime = self.__karcher_mean(xt[0],np.array(xt))
            
            data.append([xy_prime,yt_prime,xt_prime])
            
        self.KX = data
        return data
            
                
                
        
    def predict(self,X_test):
        predictions = []
        for i in range(len(X_test)):
            for j in range(len(X_test[i])):
                test_grassmann = self.__stiefel_to_grassmann(self.__my_hosvd(X_test[i][j]))
                geodesic_distances = []
                for classj in range(len(self.KX)):
                    geodesic_distances.append(self.__get_prod_geod(test_grassmann,self.KX[classj]))
                class_pred = np.argmin(geodesic_distances)
                predictions.append(class_pred)
            

        return predictions